In [ ]:
from sentence_transformers import SentenceTransformer

In [2]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
docs = [
    ["I just discovered the course, can I still join?"],
    ["I just found out about this program. Can I still enroll?"],
    ["you can join the course at any point of time"]
]

vectors = []
for d in docs:
    v = embedding_model.encode(d)
    vectors.append(v)

vectors

[array([[ 9.03510451e-02, -4.22856696e-02,  4.13428359e-02,
         -2.25168392e-02,  8.22591037e-02, -4.70754802e-02,
         -2.03891192e-03,  3.05278525e-02, -5.56562021e-02,
          3.35262530e-02, -2.89197126e-03, -1.61780138e-02,
          3.52300182e-02,  1.06728720e-02,  4.28561457e-02,
          2.56021973e-03,  4.80097346e-02, -3.33814919e-02,
         -3.23886164e-02, -2.89527699e-02, -2.41320580e-02,
          1.54270716e-02, -2.48449445e-02,  3.01835127e-02,
         -4.14910950e-02,  6.65498599e-02,  2.94066053e-02,
          8.71112477e-03, -1.20314918e-02,  5.29993512e-03,
          6.79888576e-02, -3.38245369e-02,  7.79912993e-02,
         -6.71438465e-04, -1.09950750e-04, -2.76266597e-02,
         -7.76975742e-03,  8.61175265e-03,  2.73799617e-02,
          1.28756249e-02, -4.10925671e-02,  4.64550667e-02,
         -1.97823588e-02,  4.94695939e-02,  1.38017358e-02,
         -2.82890890e-02, -4.86313887e-02, -2.40617692e-02,
          7.35451467e-03,  2.34308895e-0

In [10]:
q1, q2, d = vectors
q1.shape

(1, 768)

In [11]:
q1 = q1[0]
q2 = q2[0]
d = d[0]

q1.dot(q2) # The closer to one this is the more similar question 1 and question 2 are

np.float32(0.6203525)

In [12]:
q1.dot(d)

np.float32(0.720594)

In [13]:
q2.dot(d)

np.float32(0.48765704)

1. We take all the documents we have
2. We turn them into vectors
3. We put them in some kind of vector storage or database
4. When we have a query from a user we turn the question to a vector
5. We use the question vector to look out in the vector storage

In [16]:
import numpy as np
from tqdm.auto import tqdm
import requests

In [17]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[11]

{'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
 'section': 'General course-related questions',
 'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
d = documents[11]
text = d['question'] + ' ' + d['text']
text

"Certificate - Can I follow the course in a self-paced mode and get a certificate? No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running."

In [20]:
embeddings = []

for d in tqdm(documents):
    text = d['question'] + ' ' + d['text']
    v = embedding_model.encode(text)
    embeddings.append(v)

embeddings = np.array(embeddings)

  0%|          | 0/948 [00:00<?, ?it/s]

In [21]:
embeddings.shape

(948, 768)

In [ ]:
embeddings.dot(q1) # We get an array of similarities

array([ 4.75148976e-01,  2.49271363e-01,  6.05710268e-01,  3.01329762e-01,
        3.29438031e-01,  2.62161493e-01,  1.82255641e-01,  4.87952173e-01,
        3.87113184e-01,  9.52707976e-02,  2.88702667e-01,  3.50633085e-01,
        1.39650032e-01,  1.92375660e-01,  1.39405340e-01,  2.26143241e-01,
        2.19801277e-01,  2.36877412e-01,  2.82107055e-01,  6.11981340e-02,
        5.67240529e-02, -1.71630792e-02,  2.63339221e-01,  9.08226669e-02,
        8.90891999e-04,  1.02798216e-01,  9.89694297e-02,  2.56318986e-01,
        2.31290206e-01,  2.61218473e-02,  2.58995593e-02,  1.49240285e-01,
       -1.94128510e-02,  1.80451706e-01,  4.90299672e-01,  3.28475118e-01,
        5.79876676e-02,  8.13082904e-02,  1.01196021e-01, -1.93331093e-02,
        1.04572982e-01,  3.49248707e-01,  9.32628103e-03,  1.37938578e-02,
       -4.61915322e-02, -1.68514028e-02, -6.79952949e-02, -1.78032741e-02,
        2.49106828e-02,  1.43906921e-02,  2.52236910e-02,  6.57087266e-02,
       -1.25734955e-02,  

In [23]:
from minsearch import VectorSearch

In [24]:
vindex = VectorSearch(keyword_fields=['course'])
vindex.fit(embeddings, documents)

In [25]:
vindex.search(q1, filter_dict={'course': 'data-engineering-zoomcamp'}, num_results=5)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  's

> Check [Build Your Own Search Engine](https://github.com/alexeygrigorev/build-your-own-search-engine)

### Implementation

In [33]:
from openai import OpenAI
import json

In [30]:
openai_client = OpenAI()

In [26]:
def vector_search(question):
    # 1. Turn the query into a vector
    q = embedding_model.encode(question)

    # 2. Search
    return vindex.search(
        q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        num_results=5
    )

In [27]:
instructions = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    search_json = json.dumps(search_results)
    return prompt_template.format(
        question=question, 
        context=search_json,
    )

In [28]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [31]:
def rag(question):
    search_results = vector_search(question)
    user_prompt = build_prompt(question, search_results)
    return llm(user_prompt, instructions=instructions)

In [34]:
rag("I just found out about this program. Can I still enroll?")

"Yes, you can still enroll in the program. You don't need to register to submit homework, but be aware that there are deadlines for turning in final projects. Make sure not to leave everything for the last minute."

### Hybrid search

We can also take it one step further and combine text and vector search. This is called "hybrid search".

In [35]:
from minsearch import Index

In [36]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [37]:
def search(question):
    return index.search(
        "can i join the course now?",
        boost_dict={'question': 3.0, 'section': 0.3 },
        filter_dict={'course': 'data-engineering-zoomcamp'},
        num_results=5
    )

In [38]:
def hybrid_search(question):
    r1 = search(question)
    r2 = vector_search(question)
    return r1 + r2

In [39]:
hybrid_search("I just found out about this program. Can I still enroll?")

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin